In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import pybullet as p
import pybullet_data
import time
import math
import numpy as np
import json
import matplotlib.pyplot as plt

from scripts.buffer import ReplayBuffer
from scripts.SAC import AgentSAC
from scripts.lidar import lidar_sim
from scripts.utils import diff_drive_control, get_state, compute_reward_done, get_spawn, get_goal, setupEnvironment, dict_to_tensors
from scripts.slam import SLAM

In [3]:
# DEFINE VARIABLES AND HYPERPARAMETERS
PI = math.pi
NUM_EPISODES = 2000
ACTION_DURATION = 0.5
STEP_ACTION_DUARTION = int(ACTION_DURATION * 240.0)
MAX_EPISODE_TIME = 60
MAX_STEPS_PER_EPISODE = int(MAX_EPISODE_TIME / ACTION_DURATION)
NEW_GOAL_EVERY = 1
SAVE_EVERY = 50
DELTA_T = 0.01

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
# Load Environment
physicsClient = p.connect(p.DIRECT)
p.setGravity(0, 0, -9.8)
# p.setTimeStep(DELTA_T)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
setupEnvironment()

p.loadURDF('plane.urdf')

init_pos = [0, 0, 0]
init_orientation = p.getQuaternionFromEuler([0, 0, 0])
robotId = p.loadURDF('diff_drive_robot/diff_drive_robot.urdf', basePosition=init_pos, baseOrientation=init_orientation)
p.changeDynamics(robotId, linkIndex=-1, lateralFriction=0.0, spinningFriction=0.0, rollingFriction=0.0)
p.changeDynamics(robotId, 0, lateralFriction=5.0)
p.changeDynamics(robotId, 1, lateralFriction=5.0)
num_joints = p.getNumJoints(robotId) - 1

In [5]:
batch_size = 64
agent = AgentSAC(state_dim=495, action_dim=2).to(device)
buffer = ReplayBuffer(capacity=10000, batch_size=batch_size)
slam_node = SLAM(robotId)

reward_tracking = []
dist_goal_tracking = []
step_tracking = []

In [6]:
agent.train()
for episode in range(NUM_EPISODES):
    
    # set spawn and goal
    if episode % NEW_GOAL_EVERY == 0:
        # set spawn
        spawn = get_spawn()
        new_position = [spawn[0], spawn[1], 0.05]
        new_orientation = p.getQuaternionFromEuler([0, 0, spawn[2]])
        
        # set goal
        goal = get_goal(new_position)
    
    # reset robot pose in environment    
    p.resetBasePositionAndOrientation(robotId, posObj=new_position, ornObj=new_orientation)

    # collect initial state
    slam_node.set_robot_pose(spawn[0], spawn[1], spawn[2])
    state = get_state(robotId, goal, slam_node)
    total_reward = 0
    done = False
    print(f'Started episode, with spawn: [{spawn[0]},{spawn[1]}],  {episode}, with goal: {goal}')
    for step in range(MAX_STEPS_PER_EPISODE):
        
        # update map
        slam_node.update_map(robotId)
        
        # choose and perform action
        s_ = dict_to_tensors(state)
        action, _ = agent.actor.sample(s_['map'], s_['lidar'], s_['robot'])
        action = action.squeeze().detach().cpu().numpy()
        action_drive = diff_drive_control(linear_vel=action[0], angular_vel=action[1])
        p.setJointMotorControlArray(robotId, range(num_joints), controlMode=p.VELOCITY_CONTROL, targetVelocities=action_drive, forces=[100, 100])
        start_time = time.time()
        for _ in range(STEP_ACTION_DUARTION):
            p.stepSimulation()
            linear_vel, angular_vel = p.getBaseVelocity(robotId)
            slam_node.motion_model(linear_vel, angular_vel)
            # time.sleep(DELTA_T)

        # collect new state
        next_state = get_state(robotId, goal, slam_node)

        # get reward and done
        reward, done = compute_reward_done(state, action, next_state)
        total_reward += reward

        # store transition in buffer
        buffer.push(state, action, next_state, reward, done)

        # update model
        if len(buffer) >= batch_size:
            # sample from batch
            states, actions, next_states, rewards, dones = buffer.sample()
            # update critic 
            agent.update_critic(states, actions, next_states, rewards, dones)
    
            # update actor
            agent.update_actor(states)

        # break if done
        if done:
            break

        # update state
        state = next_state
    
    robot_pos, _ = p.getBasePositionAndOrientation(robotId)
    dist_to_goal = np.sqrt((robot_pos[0] - goal[0])**2  + (robot_pos[1] - goal[1])**2)

    # Results log
    print(f'Finished episode {episode} with total reward {total_reward}, average reward {total_reward / (step+1)}, distance to goal: {dist_to_goal}, in {step+1} steps')
    reward_tracking.append(total_reward)
    dist_goal_tracking.append(dist_to_goal)
    step_tracking.append(step)

    # Save model every few steps
    if episode % SAVE_EVERY == 0 or episode == NUM_EPISODES-1:
        torch.save(agent.state_dict(), f'saves/model_state_{episode}.pth')

# Save training data
with open('training_log/training_log_2.json', 'w') as file:
    json.dump({'Reward': reward_tracking, 'Distance': dist_goal_tracking, 'Steps': step_tracking}, file)

Started episode, with spawn: [3.0094905145237885,-10.724467817798619],  0, with goal: [8.494643302045384, -12.5]
Finished episode 0 with total reward -203.88717651278108, average reward -18.53519786479828, distance to goal: 6.036961710377423, in 11 steps
Started episode, with spawn: [2.9635789361151694,-1.4785053385850944],  1, with goal: [7.306761470526781, -3.0126816975780595]
Finished episode 1 with total reward -216.43289039142613, average reward -3.13670855639748, distance to goal: 4.893303707777274, in 69 steps
Started episode, with spawn: [2.0455065644205312,1.1141735575356932],  2, with goal: [2.1001160861892187, 1.1501602247873426]
Finished episode 2 with total reward -211.19311096329753, average reward -3.5795442536152122, distance to goal: 0.5691237541753736, in 59 steps
Started episode, with spawn: [6.926048524468889,5.738159936109472],  3, with goal: [5.770044467465408, 5.54235992875241]
Finished episode 3 with total reward 89.11924011093255, average reward 1.1726215804070

KeyboardInterrupt: 

In [7]:
# Save training data
with open('training_log/training_log_2.json', 'w') as file:
    json.dump({'Reward': reward_tracking, 'Distance': dist_goal_tracking, 'Steps': step_tracking}, file)